In [1]:
"""
inpute = Raw_Data
output = frequency of each Tower during 6 AM to 23:59 PM

author = Mahdiyar/Rahmani



"""

'\ninpute = Raw_Data\noutput = frequency of each Tower during 6 AM to 23:59 PM\n\nauthor = Mahdiyar/Rahmani\n\n\n\n'

# Tune Multithreading Options

In [2]:
"""
This Part can tune Multi threading

n_workers = Number of CPU cores
memory_limit = limitation of memory for each worker

Becareful about choosing number of workers some time its better to reduce number of workers due to memory limitations

Tune partitioning or number of partitions can help multi threading process and may reduce memory overheading

"""




from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=6, 
                       threads_per_worker=2,
                       memory_limit='4GB')
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:12463 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 12 Memory: 24.00 GB


# Import Libraries

In [3]:
"""
import required library

"""


import dask.dataframe as dd
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Prepairing Dataset

In [4]:
"""
Prepairing Dataset

"""


dataframe = dd.read_csv(r'E:\Arshad\Thesis\Datasets\20000Data\Raw data\data20000fix.csv',parse_dates=['Date_Time'])

dataframe['Hour'] = dataframe.Date_Time.dt.hour  #adding Hour column in dataset
dataframe = dataframe.drop(columns='Date_Time')  #This is for lower memory usage in Graph task

dataframe = dataframe[(dataframe.Hour>=6) & (dataframe.Hour<=24)]

# Run

In [5]:
"""
Group by each anntena by Lat and Long

"""

dataframe['Latlong'] = dataframe[['Lat','Long']].apply(tuple,axis=1,meta=(None, 'object'))
dataf = dataframe.groupby(by=['Latlong','Hour'])
f = dataf.count()


In [6]:
datafinal = f.compute() #it is a method in dask for importing data in memory 
datafinal #This is for checking

ID  Lat  Long
Latlong                     Hour                
(35.194, 51.01600000000001) 7      10   10    10
                            9      62   62    62
                            10     74   74    74
                            11    122  122   122
                            12    103  103   103
...                               ...  ...   ...
(35.685, 51.42)             7       3    3     3
                            21      3    3     3
(35.699, 51.39100000000001) 20      1    1     1
(35.719, 51.404)            21      1    1     1
(35.789, 51.39)             16      8    8     8

[12175 rows x 3 columns]

In [7]:
datafinal1 = datafinal.reset_index() #reset index for adding Latlong and Hour to columns (index to columns)

In [8]:
datafinal1['Lat'] = datafinal1['Latlong'].apply(lambda x: x[0]) #departing Lat Long from tuples to list
datafinal1['Long'] = datafinal1['Latlong'].apply(lambda x: x[1]) #departing Lat Long from tuples to list
datafinal1['Time'] = datafinal1['Hour'].apply(lambda x: pd.to_datetime('2019 {}:00:00'.format(x))) #creating fake timestamps. because Kepler can not determine Hours as a slideshow 

In [9]:
datafinal1 #This is for checking

,Latlong,Hour,ID,Lat,Long,Time
0,"(35.194, 51.01600000000001)",7,10,35.194,51.016,2019-01-01 07:00:00
1,"(35.194, 51.01600000000001)",9,62,35.194,51.016,2019-01-01 09:00:00
2,"(35.194, 51.01600000000001)",10,74,35.194,51.016,2019-01-01 10:00:00
3,"(35.194, 51.01600000000001)",11,122,35.194,51.016,2019-01-01 11:00:00
4,"(35.194, 51.01600000000001)",12,103,35.194,51.016,2019-01-01 12:00:00
...,...,...,...,...,...,...
12170,"(35.685, 51.42)",7,3,35.685,51.420,2019-01-01 07:00:00
12171,"(35.685, 51.42)",21,3,35.685,51.420,2019-01-01 21:00:00
12172,"(35.699, 51.39100000000001)",20,1,35.699,51.391,2019-01-01 20:00:00
12173,"(35.719, 51.404)",21,1,35.719,51.404,2019-01-01 21:00:00


In [10]:
datafinal1['ID'].sum() #This is for checking

28345917

In [11]:
datafinal1.drop(columns=['Latlong','Hour'],inplace=True)
datafinal1.rename(columns = {'ID':'Count'},inplace =True)

# Output

In [ ]:
"""
Output = datafinal1 which is contain Antenna lat-long and frequency of each antenna from 6AM to 23:59PM

"""

import this df to this url:https://kepler.gl/demo

In [13]:
datafinal1.to_csv('E:trial4.csv',index=False)

# Kepler GL

In [16]:
from keplergl import KeplerGl

In [17]:
df = pd.read_csv('E:trial4.csv')

In [18]:
# Change to map as you want
# map_1.config
# save or copy the current config
map_1 = KeplerGl(height=800)
map_1.add_data(data=df, name='data_1')
# Change to map as you want (Defining the config)
# map_1.config
# Copy paste the values into the next variable (config)

config = {'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': ['data_1'],
     'id': '8a5i98309',
     'name': ['Time'],
     'type': 'timeRange',
     'value': [1546343718000, 1546347197000],
     'enlarged': True,
     'plotType': 'histogram',
     'yAxis': None}],
   'layers': [{'id': '455z94m',
     'type': 'geojson',
     'config': {'dataId': 'data_1',
      'label': 'new layer',
      'color': [183, 136, 94],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': None,
       'colorRange': {'name': 'ColorBrewer YlOrRd-6',
        'type': 'sequential',
        'category': 'ColorBrewer',
        'colors': ['#ffffb2',
         '#fed976',
         '#feb24c',
         '#fd8d3c',
         '#f03b20',
         '#bd0026']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 22.4,
       'stroked': True,
       'filled': True,
       'enable3d': True,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'REGION_y', 'type': 'integer'},
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'heightField': {'name': 'REGION_y', 'type': 'integer'},
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'data_1': [{'name': 'Unnamed: 0',
        'format': None},
       {'name': 'index', 'format': None},
       {'name': 'REGION_x', 'format': None},
       {'name': 'NAME_MAHAL', 'format': None},
       {'name': 'CODE', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 24,
   'dragRotate': True,
   'latitude': 35.73926429865411,
   'longitude': 51.23263549178956,
   'pitch': 50,
   'zoom': 9.425885617522464,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [19]:
map_1.config = config

In [20]:
# Saving the map using two methods
map_1.save_to_html(file_name='E:first_map_1.html')
# map_1.save_to_html(data={'data_1': df}, config=config, file_name='/home/saeedrmd/Documents/Work/Mobile Data/KeplerGL/first_map_with_data.html')

Map saved to E:first_map_1.html!


If want to load the map next time with this saved config, the easiest way to do is to save it to a file 
and use the magic command %run to load it w/o cluttering up your notebook. See below and also the documentation
guide at the beginning of the code.

Example:

In [ ]:
### Save map_1 config to a file:
# with open('/home/saeedrmd/Documents/Work/Mobile Data/KeplerGL/config.py', 'w') as f:
#   f.write('config = {}'.format(map_1.config))

### load the config
%run /home/saeedrmd/Documents/Work/Mobile Data/KeplerGL/config.py (config is a presaved file)